# In-place Reversal of Linked List

For each: **problem → logic → step-by-step trace (table)** → **Python code** → **sample input & output**.
---

###   1) Reverse linked list (in-place)

#####   Problem

Given the head of a singly linked list, reverse the list in-place and return the new head.

Example: `1 → 2 → 3 → 4 → 5` → reversed → `5 → 4 → 3 → 2 → 1`.

#####   Logic (intuitive)

Keep three pointers: `prev`, `curr`, `next`. Iterate `curr` through the list; at each step:

* Save `next = curr.next`
* Point `curr.next = prev`
* Move `prev = curr`, `curr = next`
  When `curr` becomes `None`, `prev` is the new head.

#####   Step-by-step trace (example: 1→2→3)

|      Step |      prev | curr | next | List representation (links after change) |
| --------: | --------: | ---: | ---: | ---------------------------------------- |
| 0 (start) |      None |    1 |    2 | 1→2→3 (no change yet)                    |
|         1 | 1 (node1) |    2 |    3 | 1→None  2→3                              |
|         2 | 2 (node2) |    3 | None | 2→1→None  3→None                         |
|   3 (end) | 3 (node3) | None |    — | 3→2→1→None (prev is new head)            |

#####   Python code (in-place)

```python
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

def reverse_list(head: ListNode) -> ListNode:
    prev = None
    curr = head
    while curr:
        nxt = curr.next      ###   save next
        curr.next = prev     ###   reverse pointer
        prev = curr          ###   move prev forward
        curr = nxt           ###   move curr forward
    return prev

###   helpers for demonstration
def build_list(vals):
    head = None
    tail = None
    for v in vals:
        node = ListNode(v)
        if not head:
            head = tail = node
        else:
            tail.next = node
            tail = node
    return head

def to_list(head):
    out = []
    while head:
        out.append(head.val)
        head = head.next
    return out
```

#####   Sample input & output

Input: `[1,2,3,4,5]`
Call: `reverse_list(build_list([1,2,3,4,5]))`
Output (list): `[5,4,3,2,1]`

---

###   2) Rotate linked list (right rotate by k, in-place)

#####   Problem

Given head of linked list and integer `k`, rotate the list to the right by `k` places (in-place).
Example: `1→2→3→4→5`, `k=2` → `4→5→1→2→3`.

#####   Logic (intuitive)

* If list empty or k==0 → return head.
* First compute length `n` and tail node.
* Normalize `k = k % n`. If `k == 0` return head.
* To rotate right by `k`, new tail is the `(n-k-1)`-th node (0-indexed) and new head is `(n-k)`-th node.
* Make old tail point to old head (temporarily make it circular), cut after new tail, return new head.

#####   Step-by-step trace (example: 1→2→3→4→5, k=2)

|  Step | Action                                                            | Result                                     |
| ----: | ----------------------------------------------------------------- | ------------------------------------------ |
|     1 | compute n=5 and tail=5                                            | tail.next = None                           |
|     2 | k = 2 % 5 = 2                                                     | need new head at position 5-2 = 3 (node 4) |
|     3 | link tail→head (make circular): 5→1                               | circular list                              |
|     4 | find new tail node (node at index 2 => value 3), new head = node4 | cut new_tail.next = None                   |
| Final | Return head = node4                                               | 4→5→1→2→3→None                             |

#####   Python code (in-place)

```python
def rotate_right(head: ListNode, k: int) -> ListNode:
    if not head or not head.next or k == 0:
        return head

    ###   get length and tail
    tail = head
    n = 1
    while tail.next:
        tail = tail.next
        n += 1

    k %= n
    if k == 0:
        return head

    ###   make circular
    tail.next = head

    ###   find new tail: move (n-k-1) steps from head
    steps_to_new_tail = n - k - 1
    new_tail = head
    for _ in range(steps_to_new_tail):
        new_tail = new_tail.next

    new_head = new_tail.next
    new_tail.next = None
    return new_head
```

#####   Sample input & output

Input: `[1,2,3,4,5]`, `k=2`
Call: `rotate_right(build_list([1,2,3,4,5]), 2)`
Output: `[4,5,1,2,3]`

Edge cases: `k` may be larger than `n` (we used `%`). Empty list or one-node list handled.

---

###   3) Reorder list (L0→Ln→L1→Ln-1→L2→Ln-2 ...), in-place

#####   Problem

Given head of singly linked list, reorder it in-place so nodes are arranged in the pattern:
`L0 → Ln → L1 → Ln-1 → L2 → Ln-2 → ...`
Example: `1→2→3→4→5` → `1→5→2→4→3`.

#####   Logic (common three-step approach)

1. **Find middle**: use slow/fast pointers to find middle (`slow` stops at middle).
2. **Reverse second half**: reverse list starting at `slow.next`.
3. **Merge the two halves**: alternately take nodes from first half and reversed second half.

All done in-place (relinking pointers only).

#####   Step-by-step trace (example: 1→2→3→4→5)

1. Find middle → `slow` at `3`. Split into:

   * first: `1→2→3`
   * second: `4→5`
2. Reverse second → `5→4`
3. Merge:

   * take 1, then 5 → `1→5`
   * take 2, then 4 → `1→5→2→4`
   * leftover `3` appended → `1→5→2→4→3`

######   Trace table (detailed)

| Phase       | Action         |     First half | Second half (reversed) | Resulting merge (progress) |
| ----------- | -------------- | -------------: | ---------------------- | -------------------------- |
| find mid    | slow->3        |          1→2→3 | 4→5                    | -                          |
| reverse     | reverse 4→5    |          1→2→3 | 5→4                    | -                          |
| merge step1 | pick 1, then 5 | (next first:2) | (next second:4)        | 1→5                        |
| merge step2 | pick 2, then 4 | (next first:3) | (second exhausted)     | 1→5→2→4                    |
| finish      | append 3       |    remaining 3 | none                   | 1→5→2→4→3                  |

#####   Python code (in-place)

```python
def reorder_list(head: ListNode) -> None:
    """
    Reorders list in-place. Does not return head (mutates).
    """
    if not head or not head.next or not head.next.next:
        return

    ###   1. find middle (slow will be at mid)
    slow, fast = head, head
    while fast.next and fast.next.next:
        slow = slow.next
        fast = fast.next.next

    ###   2. reverse second half starting at slow.next
    second = slow.next
    slow.next = None  ###   cut first half
    prev = None
    curr = second
    while curr:
        nxt = curr.next
        curr.next = prev
        prev = curr
        curr = nxt
    ###   prev is head of reversed second half
    second = prev

    ###   3. merge first (head) and second alternately
    first = head
    while second:
        tmp1 = first.next
        tmp2 = second.next

        first.next = second
        second.next = tmp1

        first = tmp1 if tmp1 else None
        second = tmp2
```

#####   Sample input & output

Input: `[1,2,3,4,5]`
Call:

```python
h = build_list([1,2,3,4,5])
reorder_list(h)
to_list(h)
```

Output: `[1,5,2,4,3]`

Edge cases: even-length lists (e.g., `[1,2,3,4]` → `[1,4,2,3]`) work fine. Function mutates the original list and does not allocate new nodes.

---

###   Full runnable example (all three) — paste & run

```python
###   include ListNode, build_list, to_list from above

###   Reverse
h1 = build_list([1,2,3,4,5])
rev = reverse_list(h1)
print("reverse:", to_list(rev))  ###   [5,4,3,2,1]

###   Rotate
h2 = build_list([1,2,3,4,5])
rot = rotate_right(h2, 2)
print("rotate k=2:", to_list(rot))  ###   [4,5,1,2,3]

###   Reorder
h3 = build_list([1,2,3,4,5])
reorder_list(h3)
print("reorder:", to_list(h3))  ###   [1,5,2,4,3]
```

---

If you want, I can:

* provide the same but for **doubly-linked** lists,
* give a visual ASCII animation of pointer changes,
* or convert these into unit tests / type-hinted library-style functions.

Which one next? 😊


# Cyclic Sort  


For each:
👉 **Problem → Logic → Step-by-Step Trace (Table) → Python Code → Sample Input & Output**

---

###   🌀 1. Cyclic Sort (Base Pattern)

####   🧩 Problem

Given an array of numbers containing integers from `1` to `n` (each number appears once), sort it **in-place** without using extra space or the sort() function.

Example:
Input → `[3, 1, 5, 4, 2]`
Output → `[1, 2, 3, 4, 5]`

---

####   💡 Logic (Intuition)

* Ideal position of a number `num` is at index `num - 1`.
* For each index `i`:

  * If `nums[i]` is not at the correct position (`nums[i] != nums[nums[i]-1]`), swap it with the number at its correct index.
  * Else, move `i` forward.
* Repeat until the entire array is traversed.

✅ Runs in **O(n)** time and **O(1)** space.

---

####   🔍 Step-by-Step Trace

Input = `[3, 1, 5, 4, 2]`

| Step |   i | nums        | nums[i] | Correct idx (nums[i]-1) | Action                  |
| ---: | --: | ----------- | ------: | ----------------------: | ----------------------- |
|    1 |   0 | [3,1,5,4,2] |       3 |                       2 | swap(0,2) → [5,1,3,4,2] |
|    2 |   0 | [5,1,3,4,2] |       5 |                       4 | swap(0,4) → [2,1,3,4,5] |
|    3 |   0 | [2,1,3,4,5] |       2 |                       1 | swap(0,1) → [1,2,3,4,5] |
|    4 |   0 | [1,2,3,4,5] |       1 |                       0 | correct → i=1           |
|    5 | 1-4 | all correct |       — |                       — | end                     |

✅ Sorted: `[1, 2, 3, 4, 5]`

---

####   💻 Python Code

```python
def cyclic_sort(nums):
    i = 0
    while i < len(nums):
        correct_idx = nums[i] - 1
        if nums[i] != nums[correct_idx]:
            nums[i], nums[correct_idx] = nums[correct_idx], nums[i]
        else:
            i += 1
    return nums
```

####   🧠 Sample Input & Output

Input → `[3, 1, 5, 4, 2]`
Output → `[1, 2, 3, 4, 5]`

---

###   ❓ 2. Find Missing Numbers (LeetCode 448)

####   🧩 Problem

Given an array `nums` of size `n` where numbers are between `1` and `n`, some numbers appear once, others are missing.
Find all missing numbers **without extra space**.

Example:
Input → `[4,3,2,7,8,2,3,1]`
Output → `[5,6]`

---

####   💡 Logic

1. Use Cyclic Sort to place each number in its correct position (`nums[i] == i+1`).
2. After sorting, iterate — any index `i` where `nums[i] != i+1` means `(i+1)` is missing.

---

####   🔍 Step-by-Step Trace

Input → `[4,3,2,7,8,2,3,1]`

|    Step |   i | nums                           | Action                               |
| ------: | --: | ------------------------------ | ------------------------------------ |
|       1 |   0 | [4,3,2,7,8,2,3,1]              | swap(0,3) → [7,3,2,4,8,2,3,1]        |
|       2 |   0 | [7,3,2,4,8,2,3,1]              | swap(0,6) → [3,3,2,4,8,2,7,1]        |
|       3 |   0 | [3,3,2,4,8,2,7,1]              | swap(0,2) → [2,3,3,4,8,2,7,1]        |
|       4 |   0 | [2,3,3,4,8,2,7,1]              | swap(0,1) → [3,2,3,4,8,2,7,1]        |
|       5 | ... | eventually → [1,2,3,4,3,2,7,8] | cyclic sorted                        |
| Missing |   — | —                              | indices (4,5) → values (5,6) missing |

✅ Missing → `[5,6]`

---

####   💻 Python Code

```python
def find_disappeared_numbers(nums):
    i = 0
    while i < len(nums):
        correct_idx = nums[i] - 1
        if nums[i] != nums[correct_idx]:
            nums[i], nums[correct_idx] = nums[correct_idx], nums[i]
        else:
            i += 1

    missing = []
    for i in range(len(nums)):
        if nums[i] != i + 1:
            missing.append(i + 1)
    return missing
```

####   🧠 Sample Input & Output

Input → `[4,3,2,7,8,2,3,1]`
Output → `[5,6]`

---

###   🔁 3. Find Duplicate Numbers (LeetCode 442 / 287)

####   🧩 Problem

Given an array of integers where `1 ≤ nums[i] ≤ n`, some elements appear twice and others once.
Find all duplicates **in-place**.

Example:
Input → `[4,3,2,7,8,2,3,1]`
Output → `[2,3]`

---

####   💡 Logic

Use **Cyclic Sort** to rearrange numbers.
After rearrangement, any number that is **not in its correct index** (and duplicates remain) is a duplicate.

Steps:

1. For each `i`, place `nums[i]` at its correct index (`nums[i]-1`) if not already there.
2. When we find `nums[i] == nums[correct_idx]` (but i ≠ correct_idx) → duplicate.

---

####   🔍 Step-by-Step Trace

Input → `[4,3,2,7,8,2,3,1]`

|  Step |   i |       nums[i] |   correct_idx | Action                              |
| ----: | --: | ------------: | ------------: | ----------------------------------- |
|     1 |   0 |             4 |             3 | swap(0,3) → [7,3,2,4,8,2,3,1]       |
|   ... | ... |           ... |           ... | after full pass → [1,2,3,4,3,2,7,8] |
| Check | i=4 | nums[4]=3 ≠ 5 | duplicate = 3 |                                     |
| Check | i=5 | nums[5]=2 ≠ 6 | duplicate = 2 |                                     |

✅ Duplicates → `[3,2]`

---

####   💻 Python Code

```python
def find_duplicates(nums):
    i = 0
    while i < len(nums):
        correct_idx = nums[i] - 1
        if nums[i] != nums[correct_idx]:
            nums[i], nums[correct_idx] = nums[correct_idx], nums[i]
        else:
            i += 1

    duplicates = []
    for i in range(len(nums)):
        if nums[i] != i + 1:
            duplicates.append(nums[i])
    return list(set(duplicates))
```

####   🧠 Sample Input & Output

Input → `[4,3,2,7,8,2,3,1]`
Output → `[2,3]`

---

###   ⚙️ Summary Table

| Problem             | Key idea                                                | Output example | Time | Space |
| ------------------- | ------------------------------------------------------- | -------------- | ---- | ----- |
| **Cyclic Sort**     | Place `num` at `num-1`                                  | `[1,2,3,4,5]`  | O(n) | O(1)  |
| **Missing Numbers** | After cyclic sort, collect indices where `nums[i]!=i+1` | `[5,6]`        | O(n) | O(1)  |
| **Duplicates**      | Same as above but collect values mismatched             | `[2,3]`        | O(n) | O(1)  |

---

Would you like me to add one more **Cyclic Sort variant** — “Find the Single Missing Number (0 to n)” or “Find All Missing + Duplicate pairs”?


# Tree BFS / DFS	Level order traversal, path sums, subtree problems

Beautiful 🌳 — you’re now stepping into one of the **core DSA patterns** for trees:
**BFS (Breadth-First Search)** and **DFS (Depth-First Search)**.

Let’s cover **3 must-know problems** that represent how BFS and DFS patterns apply in trees:

1. **Level Order Traversal (BFS)**
2. **Path Sum (DFS)**
3. **Subtree Problem (Check if one tree is a subtree of another)**

For each:
👉 **Problem → Logic → Step-by-step trace (table) → Python code → Sample Input & Output**

---

##  🌿 1) Level Order Traversal (BFS)

#####    🧩 Problem

Given a binary tree, return the **level order traversal** of its nodes' values.
That means — traverse the tree **level by level**, left to right.

Example:
Input tree:

```
        1
       / \
      2   3
     / \   \
    4   5   6
```

Output: `[[1], [2,3], [4,5,6]]`

---

#####    💡 Logic (BFS)

* Use a **queue** (FIFO).
* Push root node → while queue not empty:

  * For each level, record all nodes, then enqueue their left and right children.
* Continue until queue is empty.

✅ Pattern: Level-by-level traversal (BFS).

---

#####    🔍 Step-by-step trace

| Step | Queue (front→back)              | Current Level | Output              |
| ---: | ------------------------------- | ------------- | ------------------- |
|    1 | [1]                             | []            | []                  |
|    2 | pop(1) → add children (2,3)     | [1]           | [[1]]               |
|    3 | pop(2,3) → add children (4,5,6) | [2,3]         | [[1],[2,3]]         |
|    4 | pop(4,5,6)                      | [4,5,6]       | [[1],[2,3],[4,5,6]] |

---

#####    💻 Python Code

```python
from collections import deque

class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

def level_order(root):
    if not root:
        return []
    result = []
    queue = deque([root])
    while queue:
        level = []
        for _ in range(len(queue)):
            node = queue.popleft()
            level.append(node.val)
            if node.left:
                queue.append(node.left)
            if node.right:
                queue.append(node.right)
        result.append(level)
    return result
```

#####    🧠 Sample Input & Output

Input tree: `1,2,3,4,5,None,6`
Output → `[[1], [2,3], [4,5,6]]`

---

##  🌲 2) Path Sum (DFS)

#####    🧩 Problem

Given the root of a binary tree and an integer target sum, determine if the tree has a **root-to-leaf path** such that adding up all the values equals the target sum.

Example:

```
      5
     / \
    4   8
   /   / \
 11  13  4
 / \       \
7   2       1
```

Target = 22 → True (path 5→4→11→2 = 22)

---

#####    💡 Logic (DFS)

* Use recursion (DFS).
* At each node:

  * Subtract current node’s value from `target`.
  * If leaf node and `target == 0` → found path.
  * Else, check left and right recursively.

✅ Pattern: Root-to-leaf DFS.

---

#####    🔍 Step-by-step trace

| Node | Remaining Sum | Action                      |
| ---: | ------------: | --------------------------- |
|    5 |       22-5=17 | go left (4)                 |
|    4 |       17-4=13 | go left (11)                |
|   11 |       13-11=2 | go left (7), then right (2) |
|    7 |        2-7=-5 | no path                     |
|    2 |         2-2=0 | ✅ found path (return True)  |

---

#####    💻 Python Code

```python
def has_path_sum(root, target_sum):
    if not root:
        return False
    ##  If leaf node
    if not root.left and not root.right:
        return target_sum == root.val
    ##  Otherwise continue DFS
    return (has_path_sum(root.left, target_sum - root.val) or
            has_path_sum(root.right, target_sum - root.val))
```

#####    🧠 Sample Input & Output

Input tree: `[5,4,8,11,None,13,4,7,2,None,None,None,1]`
Target = `22`
Output → `True`

---

##  🌳 3) Subtree of Another Tree (DFS)

#####    🧩 Problem

Given the roots of two binary trees `root` and `subRoot`, return `True` if `subRoot` is a **subtree** of `root`, otherwise `False`.

Example:

```
root:         subRoot:
    3             4
   / \           / \
  4   5         1   2
 / \
1   2
```

✅ subRoot is indeed a subtree of root.

---

#####    💡 Logic (DFS)

* Traverse `root` (DFS).
* For every node:

  * If `root.val == subRoot.val`, check if both trees are identical using a helper `is_same_tree`.
* Else, continue search in left and right subtree.

✅ Pattern: DFS + Recursion + Tree Equality Check.

---

#####    🔍 Step-by-step trace

|   Step |                Root Node | Compare with subRoot | Action                 |
| -----: | -----------------------: | -------------------- | ---------------------- |
|      1 |                        3 | 3≠4                  | move to left (4)       |
|      2 |                        4 | 4==4                 | check subtree equality |
|      3 | is_same_tree(4,4) → True | ✅ found              |                        |
| Output |                        — | —                    | True                   |

---

#####    💻 Python Code

```python
def is_same_tree(p, q):
    if not p and not q:
        return True
    if not p or not q or p.val != q.val:
        return False
    return is_same_tree(p.left, q.left) and is_same_tree(p.right, q.right)

def is_subtree(root, subRoot):
    if not root:
        return False
    if is_same_tree(root, subRoot):
        return True
    return is_subtree(root.left, subRoot) or is_subtree(root.right, subRoot)
```

#####    🧠 Sample Input & Output

Input:

```
root = [3,4,5,1,2]
subRoot = [4,1,2]
```

Output → `True`

---

####    🌟 Summary Table

| Problem                   | Type | Logic Pattern                | Key Data Structure | Time   | Space |
| ------------------------- | ---- | ---------------------------- | ------------------ | ------ | ----- |
| **Level Order Traversal** | BFS  | Queue (FIFO)                 | `deque`            | O(n)   | O(n)  |
| **Path Sum**              | DFS  | Recursive path sum           | recursion stack    | O(n)   | O(h)  |
| **Subtree Check**         | DFS  | Compare subtrees recursively | recursion          | O(n·m) | O(h)  |

---

Would you like me to extend this next with **Tree DFS variants** like:

* **All Root-to-Leaf Paths**
* **Maximum Path Sum**
* **Lowest Common Ancestor (LCA)**
  (all of which use DFS recursion patterns)?


Excellent 🌳 — now we’ll complete your **Tree DFS mastery** with 3 powerful and common interview problems that build directly on DFS traversal patterns:

1. **All Root-to-Leaf Paths**
2. **Maximum Path Sum**
3. **Lowest Common Ancestor (LCA)**

For each:
👉 **Problem → Logic → Step-by-step Trace → Python Code → Sample Input & Output**

---

###   🌿 1. All Root-to-Leaf Paths (DFS)

####   🧩 Problem

Given a binary tree, return all **root-to-leaf paths** as strings.

Example:
Input:

```
    1
   / \
  2   3
   \
    5
```

Output: `["1->2->5", "1->3"]`

---

####   💡 Logic

Use **DFS recursion**:

* Keep track of the path from the root.
* When a **leaf node** is reached, append the current path to result.
* Recurse left and right with the updated path.

✅ Pattern: Root-to-leaf recursion + path accumulation.

---

####   🔍 Step-by-Step Trace

| Step |          Node | Current Path | Action          |
| ---: | ------------: | ------------ | --------------- |
|    1 |             1 | "1"          | Go left (2)     |
|    2 |             2 | "1->2"       | Go right (5)    |
|    3 |             5 | "1->2->5"    | Leaf → add path |
|    4 | backtrack → 1 | "1"          | Go right (3)    |
|    5 |             3 | "1->3"       | Leaf → add path |

✅ Result = `["1->2->5", "1->3"]`

---

####   💻 Python Code

```python
def binary_tree_paths(root):
    def dfs(node, path, result):
        if not node:
            return
        path += str(node.val)
        if not node.left and not node.right:
            result.append(path)
        else:
            dfs(node.left, path + "->", result)
            dfs(node.right, path + "->", result)
    result = []
    dfs(root, "", result)
    return result
```

####   🧠 Sample Input & Output

Input: `[1,2,3,None,5]`
Output: `["1->2->5", "1->3"]`

---

###   🌲 2. Maximum Path Sum (DFS)

####   🧩 Problem

Given a binary tree, find the **maximum path sum** (sum of node values along any path).
A path may start and end at any node.

Example:

```
      -10
      /  \
     9   20
        /  \
       15   7
```

Output: `42` (15 → 20 → 7)

---

####   💡 Logic

Use **DFS (postorder)**:

* For each node, compute:

  * `left_gain = max(0, dfs(left))`
  * `right_gain = max(0, dfs(right))`
  * **Current path sum** = `node.val + left_gain + right_gain`
* Update global max.
* Return `node.val + max(left_gain, right_gain)` (the best single path up).

✅ Pattern: DFS postorder recursion, propagate best path sum up the tree.

---

####   🔍 Step-by-Step Trace

| Node | left_gain | right_gain | Path Sum | Max So Far |
| ---: | --------: | ---------: | -------: | ---------: |
|   15 |         0 |          0 |       15 |         15 |
|    7 |         0 |          0 |        7 |         15 |
|   20 |        15 |          7 |       42 |     **42** |
|    9 |         0 |          0 |        9 |         42 |
|  -10 |         9 |         42 |       41 |         42 |

✅ Max path sum = `42`

---

####   💻 Python Code

```python
def max_path_sum(root):
    max_sum = float('-inf')

    def dfs(node):
        nonlocal max_sum
        if not node:
            return 0
        left_gain = max(dfs(node.left), 0)
        right_gain = max(dfs(node.right), 0)
        current_sum = node.val + left_gain + right_gain
        max_sum = max(max_sum, current_sum)
        return node.val + max(left_gain, right_gain)

    dfs(root)
    return max_sum
```

####   🧠 Sample Input & Output

Input: `[-10,9,20,None,None,15,7]`
Output: `42`


 **Step-by-Step Execution**
- Start at node -10:
    - Recurse left → node 9 → no children → returns 9
    - Recurse right → node 20
- At node 20:
    - Recurse left → node 15 → no children → returns 15
    - Recurse right → node 7 → no children → returns 7
    - current_sum = 20 + 15 + 7 = 42 → update max_sum = 42
    - Return to parent: 20 + max(15, 7) = 35
- Back at node -10:
    - Left gain = 9
    - Right gain = 35
    - current_sum = -10 + 9 + 35 = 34 → max_sum remains 42
- Return to parent: -10 + max(9, 35) = 25




---

###   🌳 3. Lowest Common Ancestor (LCA)

####   🧩 Problem

Given a binary tree and two nodes `p` and `q`, find their **Lowest Common Ancestor (LCA)** —
the lowest node that has both `p` and `q` as descendants.

Example:

```
        3
       / \
      5   1
     / \ / \
    6  2 0  8
      / \
     7   4
```

For `p=5, q=1`, LCA = `3`
For `p=5, q=4`, LCA = `5`

---

####   💡 Logic

Use **DFS recursion**:

* If current node is `None` → return `None`
* If node == `p` or `q` → return node
* Recurse left and right

  * If both sides return non-`None`, current node is LCA
  * Else, return the non-`None` child.

✅ Pattern: DFS returning node references.

---

####   🔍 Step-by-Step Trace (p=5, q=1)

| Node |                         Left | Right | Return |
| ---: | ---------------------------: | ----: | -----: |
|    5 |                      found p |     - |      5 |
|    1 |                      found q |     - |      1 |
|    3 | left=5, right=1 → both found | **3** |        |

✅ LCA = 3

---

####   💻 Python Code

```python
def lowest_common_ancestor(root, p, q):
    if not root or root == p or root == q:
        return root
    left = lowest_common_ancestor(root.left, p, q)
    right = lowest_common_ancestor(root.right, p, q)
    if left and right:
        return root
    return left if left else right
```

####   🧠 Sample Input & Output

Input:

```
root = [3,5,1,6,2,0,8,None,None,7,4]
p = 5, q = 1
```

Output: `3`

---

###   🌟 Summary Table

| Problem                          | Pattern | Traversal | Key Operation     | Time | Space |
| -------------------------------- | ------- | --------- | ----------------- | ---- | ----- |
| **All Root-to-Leaf Paths**       | DFS     | Preorder  | Path building     | O(n) | O(h)  |
| **Maximum Path Sum**             | DFS     | Postorder | Global max update | O(n) | O(h)  |
| **LCA (Lowest Common Ancestor)** | DFS     | Preorder  | Return node ref   | O(n) | O(h)  |

---

✅ **DFS Takeaways**

* DFS can be **preorder**, **inorder**, or **postorder** — depends on when you process the node.
* Most **path**, **sum**, and **ancestor** problems rely on **recursive DFS**.
* BFS (queue) is ideal for **level-based** logic; DFS (recursion/stack) is ideal for **path-based** logic.

---

Would you like me to move next into **Binary Search Tree (BST)-specific DFS/BFS problems** (like insertion, validation, k-th smallest, LCA in BST)?


Great observation, Rahul — let’s walk through the **full execution trace**, including **non-matching nodes** and how they contribute to the recursion. This will clarify how the function handles every node, not just the ones that match `p` or `q`.

---

####  🌳 Tree Recap

```
        7
       / \
      4   10
         /  \
        9    12
       /    /  \
      8    11  13
```

We’re finding the **LCA of nodes 8 and 11**.

---

####  🧠 Full Execution Trace (with all nodes)

```python
def lowest_common_ancestor(root, p, q):
    if not root or root == p or root == q:
        return root
    left = lowest_common_ancestor(root.left, p, q)
    right = lowest_common_ancestor(root.right, p, q)
    if left and right:
        return root
    return left if left else right
```

---

####  🔍 Step-by-Step

#####  1. **Start at root 7**
- 7 ≠ 8 or 11 → recurse left and right

#####  2. **Left subtree: node 4**
- 4 ≠ 8 or 11
- 4 has no children → both `left` and `right` return `None`
- So `lowest_common_ancestor(4, 8, 11)` returns `None`

#####  3. **Right subtree: node 10**
- 10 ≠ 8 or 11 → recurse left and right

#####  4. **Left of 10: node 9**
- 9 ≠ 8 or 11 → recurse left and right

######  → Left of 9: node 8
- 8 == p → return node 8

######  → Right of 9: None
- return `None`

- So `lowest_common_ancestor(9, 8, 11)` returns node 8

#####  5. **Right of 10: node 12**
- 12 ≠ 8 or 11 → recurse left and right

######  → Left of 12: node 11
- 11 == q → return node 11

######  → Right of 12: node 13
- 13 ≠ 8 or 11 → no children → returns `None`

- So `lowest_common_ancestor(12, 8, 11)` returns node 11

#####  6. **Back at node 10**
- Left returned 8, right returned 11 → both non-null → return node 10

#####  7. **Back at root 7**
- Left returned `None`, right returned node 10 → return node 10

---

####  ✅ Final Result

```python
lowest_common_ancestor(root, node8, node11) → 10
```

---

####  🔄 Summary of Non-Matching Nodes

| Node | Action Taken |
|------|--------------|
| 4    | No match → returns `None` |
| 9    | No match → returns result from child (8) |
| 12   | No match → returns result from child (11) |
| 13   | No match → returns `None` |
| 7    | No match → returns result from right (10) |

---

This trace shows how **every node is visited**, and how **non-matching nodes either return `None` or propagate results from children**. Want me to wrap this into a visual debugger or add logging to each step for your toolkit?
